## Grundlagen Maschineller Lernverfahren | ML_INF19A | 2021
**Datum: 11.11.2021**

# Über- & Unteranpassung

In [ ]:
# Lade Bibliotheken
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from sklearn.datasets import make_moons

# Erstelle Datensatz: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_moons.html
data = make_moons(n_samples = 500, noise = 0.15, random_state = 2021)

X = data[0]
y = data[1]

In [ ]:
# Zeichne Datenpunkte
plt.figure(figsize=(15,9))
plt.plot(X[:,0][y==0],X[:,1][y==0],'s')
plt.plot(X[:,0][y==1],X[:,1][y==1],'o')

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# Transformiere Inputdaten für Modell in ein Polynom 
poly_features_1 = PolynomialFeatures(degree = 3) # Grad = 3
Xpoly_1 = poly_features_1.fit_transform(X)

poly_features_2 = PolynomialFeatures(degree = 28) # Grad = 28
Xpoly_2 = poly_features_2.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split

# Splitte Datenmenge für Training und Test

# Originaldaten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2021)

# Polynomiell-transformierte Daten #1
Xpoly1_train, Xpoly1_test, ypoly1_train, ypoly1_test = train_test_split(Xpoly_1, y, test_size = 0.2, random_state = 2021)

# Polynomiell-transformierte Daten #2
Xpoly2_train, Xpoly2_test, ypoly2_train, ypoly2_test = train_test_split(Xpoly_2, y, test_size = 0.2, random_state = 2021)

In [ ]:
from sklearn.linear_model import LogisticRegression

# Erstelle eine Instanz für das LogReg Modell OHNE Regularisierung (-> penalty = "none")
# Hinweis: Beachte Kombination aus PENALTY und SOLVER -> vgl. https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

log_reg   = LogisticRegression(penalty="none", solver="saga", max_iter=5000) # für originale Daten
log_reg_1 = LogisticRegression(penalty="none", solver="saga", max_iter=5000) # für polynomiell transformierte Daten #1
log_reg_2 = LogisticRegression(penalty="none", solver="saga", max_iter=5000) # für polynomiell transformierte Daten #2

In [ ]:
# Trainiere die Modelle mit den jeweiligen Datensätzen
log_reg.fit(X_train, y_train)
log_reg_1.fit(Xpoly1_train, ypoly1_train)
log_reg_2.fit(Xpoly2_train, ypoly2_train)

In [ ]:
# Klassifikationszonen visualisieren

# Idee: Lege ein Punktenetz an und lass für jeden der Punkte die Klasse durch das Modell bestimmen.
# Dadurch ergibt sich ein "Karte" die anzeigt welche Datenpunkte zu welcher Klasse sortiert werden.

# Lege gleichmäßig verteilte Punkte für x1 und x2 Achse an
x1_mesh = np.linspace(-1.5, 2.5, 500) # x1-Achse
x2_mesh = np.linspace(-1.5, 2.5, 500) # x2-Achse

# Füge beide Punkteverteilungen zu einem "2D Netz" zusammen
x0, x1 = np.meshgrid(x1_mesh, x2_mesh)
X_mesh = np.c_[x0.ravel(), x1.ravel()]

# Vorhersage treffen auf Punktenetz
y_pred = log_reg.predict(X_mesh).reshape(x0.shape)
y_pred_1 = log_reg_1.predict(poly_features_1.fit_transform(X_mesh)).reshape(x0.shape)
y_pred_2 = log_reg_2.predict(poly_features_2.fit_transform(X_mesh)).reshape(x0.shape)


## Unteranpassung

 Linear

In [ ]:
# Datensatz darstellen
plt.figure(figsize=(15,9))
plt.plot(X[:,0][y==0],X[:,1][y==0],'s')
plt.plot(X[:,0][y==1],X[:,1][y==1],'o')

plt.xlabel("Feature $x_1$")
plt.ylabel("Feature $x_2$")
plt.title('Unteranpassung')

# Zuordnungbereiche darstellen
plt.contourf(x0, x1, y_pred)

# Interpretation:
# Lineares Modell wird auf eine nicht-lineare Datenmenge angewendet -> Trennung ist eine Gerade


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print("Accuracy:\t", accuracy_score(y_test, log_reg.predict(X_test)))
print("Precision:\t", precision_score(y_test, log_reg.predict(X_test)))
print("Recall:\t\t", recall_score(y_test, log_reg.predict(X_test)))
print("F1 Score:\t", f1_score(y_test, log_reg.predict(X_test)))

# Überanpassung

Polynom #2

In [ ]:
# Datensatz darstellen
plt.figure(figsize=(15,9))
plt.plot(X[:,0][y==0], X[:,1][y==0],'s')
plt.plot(X[:,0][y==1], X[:,1][y==1],'o')

plt.xlabel("Feature $x_1$")
plt.ylabel("Feature $x_2$")
plt.title('Überanpassung')

# Zuordnungbereiche darstellen
plt.contourf(x0, x1, y_pred_2) # Klassifizierungszone passt sich sehr stark den Trainingsdaten an!

# Interpretation:
# Lineares Modell wird mit einem zu hohen Grad polynomiell erweitert und der nicht-lineare Datenmenge angewendet -> Trennung ist eine über-angepasste Linie (polynomielle Form)

In [ ]:
print("Accuracy:\t", accuracy_score(ypoly2_test, log_reg_2.predict(Xpoly2_test)))
print("Precision:\t", precision_score(ypoly2_test, log_reg_2.predict(Xpoly2_test)))
print("Recall:\t\t", recall_score(ypoly2_test, log_reg_2.predict(Xpoly2_test)))
print("F1 Score:\t", f1_score(ypoly2_test, log_reg_2.predict(Xpoly2_test)))

# Komplexität nichtig gewählt

Polynom #1

In [ ]:
# Datensatz darstellen
plt.figure(figsize=(15,9))
plt.plot(X[:,0][y==0],X[:,1][y==0],'s')
plt.plot(X[:,0][y==1],X[:,1][y==1],'o')

plt.xlabel("Feature $x_1$")
plt.ylabel("Feature $x_2$")
plt.title('Komplexität ausreichend')

# Zuordnungbereiche darstellen
plt.contourf(x0, x1, y_pred_1)

# Interpretation:
# Lineares Modell wird mit dem korrekten Grad der nicht-lineare Datenmenge polynomiell erweitert und angewendet -> Trennung ist eine angepasste Linie (polynomielle Form)

In [ ]:
print("Accuracy:\t", accuracy_score(ypoly1_test, log_reg_1.predict(Xpoly1_test)))
print("Precision:\t", precision_score(ypoly1_test, log_reg_1.predict(Xpoly1_test)))
print("Recall:\t\t", recall_score(ypoly1_test, log_reg_1.predict(Xpoly1_test)))
print("F1 Score:\t", f1_score(ypoly1_test, log_reg_1.predict(Xpoly1_test)))